In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [51]:
# Import the excel spreadsheets in the data folder
installer_df = pd.read_excel('../data/Installer.xlsx')
involver_df = pd.read_excel('../data/Involver.xlsx')

c:\Users\syeda\OneDrive\Documents\American Bureau of Shipping\projects\nlp_risk_prediction\nlp_env\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\syeda\OneDrive\Documents\American Bureau of Shipping\projects\nlp_risk_prediction\nlp_env\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [52]:
# Attempt to merge using a subset of key columns
merge_on_columns = ['Site', 'Vessel_Name', 'Wo_No']

df = pd.concat([installer_df, involver_df], axis=0)

feature_columns = ['Group', 'Symptom', 'Error_Cause', 'Cause_Details', 'Error_Class', 'Discovery', 'Completion_Note', 'Action_Taken', 'Work_Description', 'Directive']
target_column = 'EBS1'

# Filter the dataframe for the selected columns
df = df[feature_columns + [target_column]].dropna()

In [53]:
# Encode the target column
label_encoder = LabelEncoder()
df[target_column] = label_encoder.fit_transform(df[target_column].astype(str))

In [54]:
# Remove classes with less than 2 items
class_counts = df[target_column].value_counts()
df = df[df[target_column].isin(class_counts[class_counts >= 2].index)]

# Create the vectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=20000, stop_words='english')

X = df[feature_columns]
y = df[target_column]

# Transform the features using CountVectorizer
X_vec = vectorizer.fit_transform(X.apply(lambda x: ' '.join(x), axis=1))

# Train test split with stratification to ensure all classes are represented
X_train, X_valid, y_train, y_valid = train_test_split(X_vec, y, test_size=0.2, random_state=30, stratify=y)

# Re-encode the target variable after splitting
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)

# Create XGBoost Model
xgb_model = XGBClassifier(
    objective='multi:softmax',
    learning_rate=0.12,
    n_estimators=1400,
    max_depth=13,
    min_child_weight=1,
    gamma=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=30
)

# Eval and fitting of the initial model
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.12, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=13, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [55]:
# Evaluate on the test set
y_pred = xgb_model.predict(X_valid)
test_accuracy = accuracy_score(y_valid, y_pred)
print("Test accuracy: ", test_accuracy)

# Print the classification report
print("Classification Report:")
print(classification_report(y_valid, y_pred))

# # Define parameter grid for hyperparameter tuning
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'subsample': [0.6, 0.8, 1.0]
# }

# # Initialize GridSearchCV
# grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, scoring='accuracy', verbose=2)

# # Fit GridSearchCV
# grid_search.fit(X_train_transformed, y_train)

# # Print the best parameters and score
# print("Best parameters found: ", grid_search.best_params_)
# print("Best cross-validation accuracy: ", grid_search.best_score_)

# # Evaluate on the test set with the best model
# best_model = grid_search.best_estimator_
# y_pred_best = best_model.predict(X_test_transformed)
# test_accuracy_best = accuracy_score(y_test, y_pred_best)
# print("Test accuracy with best model: ", test_accuracy_best)

# # Print the classification report for the best model
# print("Classification Report with best model:")
# print(classification_report(y_test, y_pred_best))

Test accuracy:  0.8823529411764706
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        26
           2       0.00      0.00      0.00         1
           3       0.65      1.00      0.79        11
           4       0.96      0.96      0.96        25
           5       0.00      0.00      0.00         1
           6       0.50      1.00      0.67         1
           7       0.75      0.75      0.75         4
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         2
          12       0.80      1.00      0.89         4
          13       1.00      0.50      0.67         2
          14       0.96      0.96      0.96        26
          15       1.00      1.00      1.00         1
          16       1.00      0.25      0.40         4
          17       0.67

c:\Users\syeda\OneDrive\Documents\American Bureau of Shipping\projects\nlp_risk_prediction\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syeda\OneDrive\Documents\American Bureau of Shipping\projects\nlp_risk_prediction\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\syeda\OneDrive\Documents\American Bureau of Shipping\projects\nlp_risk_prediction\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set